# Retrieving and Post-Processing Previous Jobs from DesignSafe
Pedro Arduino - UW Computational Geomechanics Group

This notebook reads and post-processes previous jobs executed on DesignSafe, whether run through the portal or a Jupyter Notebook.

# Import required libraries

In [ ]:
# Before we start let us install a python package for plotting
try:
    import seaborn

except:
    import pip

    pip.main(["install", "--user", "seaborn", "-q"])
    pip.main(
        ["install", "--user", "matplotlib", "--upgrade", "-q"]
    )  # This is needed in order to use sb.kde
    print("********* Please restart the session ***********")
    import IPython

    IPython.Application.instance().kernel.do_shutdown(
        True
    )  # automatically restarts kernel

In [ ]:
pip install tabulate -q

In [ ]:
import tabulate
import seaborn as sns
from IPython.display import HTML, display

# Setup tapis

In [ ]:
!pip install --upgrade pip --user -q
!pip install tapipy -q
from tapipy.tapis import Tapis

In [ ]:
# Method using getpass --- PREFERRED
# =================================

from getpass import getpass

# from tapipy.tapis import Tapis

t = Tapis(
    base_url="https://designsafe.tapis.io",
    username=getpass("input username: "),
    password=getpass("input password: "),
)
t.get_tokens()

In [ ]:
from DS_GenFunctionsV3 import *  # General python functions useful for DesignSafe
import os
import uuid
from datetime import date

## Get job index

Specify the tool name and list length

In [ ]:
toolName = "opensees"
listLength = 25

In [ ]:
jobs_list = t.jobs.getJobList(
    limit=10, orderBy="lastUpdated(desc),name(asc)", computeTotal=True
)

In [ ]:
# header = ["index","name","status","created","uuid", "owner","archiveSystemId","execSystemId"]
header = ["index", "name", "status", "created", "uuid", "owner", "execSystemId"]

rows = []
count = 1
for x in jobs_list:
    row = [count]
    if x.name.startswith(toolName):
        count += 1
        for key in header[1:]:
            row += [getattr(x, key)]
        rows += [row]
        if count > listLength:
            break
# print(tabulate.tabulate(rows, header, tablefmt='rst'))
display(HTML(tabulate.tabulate(rows, header, tablefmt="html")))

print("Please select the index")

In [ ]:
idx = 1

### Identify job status and location

In [ ]:
jobid = rows[idx - 1][4]
archiveSystem = rows[idx - 1][6]
if not rows[idx - 1][2] == "FINISHED":
    raise SystemExit("The job failed or incomplete!")
# t.jobs.getJobOutputDownload(jobUuid=jobid, outputPath='/')
files = t.jobs.getJobOutputList(jobUuid=jobid, outputPath="/")
archivePath = files[0].url.split(jobid)[0] + jobid
archivePath = archivePath.split("tapis://designsafe.storage.default/")[-1]

### Go to archived folder

In [ ]:
jobFolderPath = "/home/jupyter/MyData/" + "/".join(archivePath.split("/")[1:])
inputFolderName = "/inputDirectory"
FolderPath = jobFolderPath + inputFolderName
os.chdir(FolderPath)

files = os.listdir(FolderPath)
listFiles = False

if files:
    print("Files are present in the directory.")
    if listFiles:
        for file in files:
            print(f"- {file}")
else:
    print("No files found in the directory.")

# Postprocess Results

### Import python libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### Plot acceleration response spectra

Plot acceleration response spectra on log-linear scale

In [ ]:
from plotAcc import plot_acc

plot_acc()